In [0]:
ls ../scripts/8TeV_pt_dependent/

In [0]:
from rootpy.io import root_open
from rootpy.plotting import Hist, Canvas
from ROOT import TF1, TPaveText

lumi = 19789.0
option = "first"
dir_path = "../scripts/8TeV_pt_dependent/"
tag_dict = { "CSVL": ["combinedSecondaryVertexBJetTags", 0.244 ],
             "CSVM": ["combinedSecondaryVertexBJetTags", 0.679 ],
             "CSVT": ["combinedSecondaryVertexBJetTags", 0.898 ],
             "JPL": ["jetProbabilityBJetTags", 0.275 ],
             "JPM": ["jetProbabilityBJetTags", 0.545 ],
             "JPT": ["jetProbabilityBJetTags", 0.79 ],
             "TCHPT" : ["trackCountingHighPurBJetTags", 3.41 ]}

In [0]:

c = Canvas(1000,500)

s_name = "CSVT"
tagger = tag_dict[s_name][0]
wp = tag_dict[s_name][1]

sf_array = []
sf_err_array = []

c_bins = [30, 50, 80, 120, 210, 320]
bins = np.array([30, 50, 80, 120, 210, 320])

c_hist = Hist(c_bins)
c_hist.SetStats(False)
c_hist.SetAxisRange(0.7,1.3,"Y")
c_hist.GetXaxis().SetMoreLogLabels()
c_hist.GetXaxis().SetNoExponent()
c_hist.SetXTitle("Jet p_{T}[GeV/c]")
c_hist.SetYTitle("Data/Sim. b-tag SF")
c_hist.SetTitle(s_name + " - CMS preliminary - {}".format(lumi) +" pb^{-1} at #sqrt{s} = 8 TeV ")
c_hist.Draw()

root_file = root_open("{}fit_result_{}_{}_{}.root".format(dir_path, tagger, wp, option))
fit_result = root_file.fitresult_sim_kin_pdf_data_hist
i_pars = fit_result.floatParsInit()
f_pars = fit_result.floatParsFinal()
hist = Hist( bins, title = "FtCM")

ttbar_xs = f_pars[f_pars.index("xsec_TTbar_Summer13")].getVal()
ttbar_xs_err = f_pars[f_pars.index("xsec_TTbar_Summer13")].getError()
print "{} & ${}\pm{}".format(ttbar_xs,ttbar_xs_err)
bkg_norm = f_pars[f_pars.index("kappa")].getVal()
bkg_norm_err = f_pars[f_pars.index("kappa")].getError()
#print "kappa {} \pm {} ".format(bkg_norm,bkg_norm_err)
text_box = TPaveText(0.60,0.7,0.87,0.87, "NDC")
text_box.AddText("KinFtCM")
text_box.AddText(" - #sigma_{t#bar{t}}" + 
                 " = {:4.0f} #pm {:2.0f}".format(ttbar_xs, ttbar_xs_err) +
                 " fb^{-1}")
text_box.AddText(" - #kappa" + 
                 " = {:5.2f} #pm {:5.2f}".format(bkg_norm,bkg_norm_err ))
text_box.Draw()

print "{} & ${:.0f} \pm {:.0f}$".format(s_name, ttbar_xs, ttbar_xs_err),


for i in range(0,len(bins)-1):
    mc_val = i_pars[f_pars.index("b_tag_eff_{}".format(i))].getVal()
    data_val = f_pars[f_pars.index("b_tag_eff_{}".format(i))].getVal()
    data_err = f_pars[f_pars.index("b_tag_eff_{}".format(i))].getError()
    #print "b-tagging efficiency {}\pm{} ".format(data_val,data_err)
    sf = data_val/mc_val
    sf_array.append(sf)
    sf_err = data_err/mc_val
    sf_err_array.append(sf_err) 
    print "& ${:.3f}\pm{:.3f}$ ".format(sf,sf_err),

    hist[i+1] = (sf,sf_err)
hist.Draw("SAME E1")

print "\\\\"



c.SetLogx(1)
c.GetPad(0).SetTickx()
c.GetPad(0).SetTicky()
t = c.GetPrimitive("title")
t.SetTextSize(0.04)
t.SetBBoxCenterX(300)
t.SetBBoxCenterY(30)
c

In [0]:
import numpy as np

def array_from_file(array_name, file_name, veto ="veto", debug = False):
    array = []
    with open(file_name, 'r') as f:
        words = f.read().split()
        for i,w in enumerate(words):
            if (array_name in w) and not (veto in w):
                if debug:
                    print "{},{},{}".format(words[i], words[i+1], words[i+2])
                if ("=" not in w):
                    if "=" is words[i+1]:
                        if (";" not in words[i+2]):
                            array.append(float(words[i+2]))
                        else:
                            array.append(float(words[i+2].split(";",1)[0]))
                    else:
                        no_equal = words[i+1].split("=",1)[1]
                        if (";" not in no_equal):
                            array.append(float(no_equal))
                        else:
                            array.append(float(no_equal.split(";",1)[0]))
                else:
                    rem_equal = w.split("=",1)[1]
                    if (";" not in rem_equal):
                        array.append(float(rem_equal))
                    else:
                        array.append(float(rem_equal.split(";",1)[0]))
                    
                        
                        
    return np.array(array)
    
        
        
print array_from_file("eff_PT","../../../../../paraPabloM/SF8TeV/CSVM.C")     

In [0]:


b_file = "../../../../../paraPabloM/SF8TeV/PtBins.C"
v_file = "../../../../../paraPabloM/SF8TeV/{}.C".format(s_name)


methods = {}

methods["Kin-FtC"] = {"x" : (bins[1:]-bins[:-1])/2+bins[:-1],
                 "x_err" : (bins[1:]-bins[:-1])/2,
                 "y" : np.array(sf_array),
                 "y_err" : np.array(sf_err_array)}

methods["LT"] = {"x" : array_from_file("x[",b_file,"ex"),
                 "x_err" : array_from_file("ex[",b_file),
                 "y" : array_from_file("sf_JP",v_file),
                 "y_err" : array_from_file("sf_eror_JP",v_file) }

methods["PtRel"] = {"x" : array_from_file("x1[",b_file,"ex")[:4],
                    "x_err" : array_from_file("ex1[",b_file)[:4],
                    "y" : array_from_file("sf_PT",v_file)[:4],
                    "y_err" : np.sqrt(array_from_file("sf_stat_PT",v_file)**2+(array_from_file("sf_PT",v_file)*array_from_file("total_syst_PT",v_file))**2)[:4]}

methods["IP3D"] = {"x" : array_from_file("x3[",b_file,"ex")[3:0],
                    "x_err" : array_from_file("ex1[",b_file)[3:0],
                    "y" : array_from_file("sf_IP",v_file)[3:0],
                    "y_err" : np.sqrt(array_from_file("sf_stat_IP",v_file)**2+(array_from_file("sf_IP",v_file)*array_from_file("total_syst_IP",v_file))**2)[3:0]}

methods["System8"] = {"x" : array_from_file("x2[",b_file,"ex2"),
                    "x_err" : array_from_file("ex2[",b_file),
                    "y" : array_from_file("sf_SY",v_file),
                    "y_err" : np.sqrt(array_from_file("sf_stat_SY",v_file)**2+(array_from_file("sf_SY",v_file)*.01*array_from_file("total_syst_SY",v_file))**2)}

methods["LT top"] = {"x" : array_from_file("x5[",b_file,"ex5"),
                    "x_err" : array_from_file("ex5[",b_file),
                    "y" : array_from_file("sf_TT",v_file),
                    "y_err" : np.sqrt(sum([ (array_from_file("sf_TT",v_file)*array_from_file(sys,v_file))**2 for sys in ["sf_stat_TT","pu_TT","model_TT","jes_TT" ]])) }

syst_PSI = ["cosalpha_PSI", "deltaR_PSI", "deltaRprobe_PSI" ,
            "lxys_PSI", "mass_PSI", "ptsum_PSI", "z0_PSI", "pu_PSI" , "bias_PSI", "cor_PSI"]
methods["LT J/Psi"] = {"x" : array_from_file("x6[",b_file,"ex6")[:-1],
                    "x_err" : np.array([5,5,5,5,5,5, 11, 9,22, 21])[:-1],
                    "y" : array_from_file("sf_PSI",v_file)[:-1],
                    "y_err" : np.sqrt(array_from_file("sf_stat_PSI",v_file)**2+sum([ (array_from_file("sf_PSI",v_file)*array_from_file(sys,v_file))**2 for sys in syst_PSI]))[:-1] }

In [0]:
array_from_file("x1[",b_file,"ex")

In [0]:
syst_PSI = ["sf_stat_PSI","cosalpha_PSI", "deltaR_PSI", "deltaRprobe_PSI" ,
            "lxys_PSI", "mass_PSI", "ptsum_PSI", "z0_PSI", "pu_PSI" , "bias_PSI", "cor_PSI"]
for sys in syst_PSI:
    print array_from_file(sys,v_file)

In [0]:
np.sqrt(sum([ array_from_file(sys,v_file)**2 for sys in syst_PSI]))

In [0]:
 array_from_file("sf_IP", "../../../../../paraPabloM/SF8TeV/CSVT.C")

In [0]:
print array_from_file("x6[",b_file,"veto", True)
print [5,5,5,5,5,5, 11, 9,22, 21]

In [0]:
print array_from_file("sf_IP",v_file,"veto", True)
print array_from_file("sf_stat_PSI",v_file,"veto", True)

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams.update({'font.size': 14})

fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)


for m_name, m_data in methods.items():
    ax.errorbar(m_data["x"],m_data["y"], xerr=m_data["x_err"], yerr=m_data["y_err"], fmt='o',label=m_name)
    
plt.legend(mode="expand", ncol=7,numpoints=1, frameon=False)
ax.set_xlim(30,320)
ax.set_ylim(0.7,1.3)
ax.semilogx()
ax.set_xticks(bins)
ax.set_xticklabels(bins)
ax.set_title("{} - CMS Preliminary - {}".format(s_name,lumi) +" $pb^{-1}$ at $\sqrt{s}$ = 8 TeV ")
ax.set_ylabel("Data/Sim. b-tag SF")
ax.set_xlabel("jet momentum \GeV")
fig.savefig("sf_comparison_8tev_{}.pdf".format(s_name))
    

In [0]:
cat ../../../../../paraPabloM/SF8TeV/CSVM.C

In [0]:
cat